In [1]:
import os
import shutil
import math
import pandas as pd
import sqlite3
import pymongo
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn import cross_validation, preprocessing, decomposition
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, recall_score, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE

/home/fytroo/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
receipts = [
    'MED_CM_TBL_2016',
    'MED_CO_TBL_2016',
    'MED_GR_TBL_2016',
    'MED_HOKO_TBL_2016',
    'MED_IY_TBL_2016',
    'MED_SI_TBL_2016',
    'MED_SJ_TBL_2016',
    'MED_SY_TBL_2016',
    'MED_TO_TBL_2016',
    'PHA_CM_TBL_2016',
    'PHA_CO_TBL_2016',
    'PHA_HOKO_TBL_2016',
    'PHA_IY_TBL_2016',
    'PHA_TO_TBL_2016']

In [3]:
client = pymongo.MongoClient('localhost', 27017)
db = client.kikin

In [4]:
dbname = 'kikin.sqlite3'
con = sqlite3.connect(dbname)
c = con.cursor()
dfs = {}
for receipt in receipts:
    q = 'select * from {}'.format(receipt)
    df = pd.io.sql.read_sql(q,con)
    dfs[receipt] = df
c.close()

### 傷病、診療、医薬から説明変数を取り出す

In [5]:
df = dfs['MED_SY_TBL_2016']
sy = df['shobyo_code'].drop_duplicates()
n_sy = df['shobyo_code'].drop_duplicates().count()
n_sy

2930

In [6]:
df = dfs['MED_SI_TBL_2016']
si = df['s_tekiyo_code'].drop_duplicates()
n_si = df['s_tekiyo_code'].drop_duplicates().count()
n_si

1371

In [7]:
df = dfs['MED_IY_TBL_2016']
iy = df['s_tekiyo_code'].drop_duplicates()
n_iy = df['s_tekiyo_code'].drop_duplicates().count()
n_iy

2908

In [8]:
dd = pd.concat([sy,iy,si])
dd.count()

7209

### 傷病コード、診療コード、医薬コードに重複なし

In [9]:
dd.drop_duplicates().count()

7209

In [10]:
dd.count()

7209

In [11]:
x_values = dd.values#.tolist()
x_size = x_values.size
x_dic = { v:k for (k,v) in enumerate(x_values)}
x_kv = { k:v for (k,v) in enumerate(x_values)}

### 素性をつくる

In [12]:
xs = []
ys = []
doc = db.med.find()
for iii, row in enumerate(doc):
    x = np.zeros(x_size)
    
    # 該当する傷病があれば、その傷病に対応するindexの値を1, なければ0
    if 'MED_SY_TBL_2016' in row:
        for d in row['MED_SY_TBL_2016']:
            idx = x_dic[d['shobyo_code']]
            x[idx] = 1
            
    # 医薬品に対応するindexに点数を挿入
    if 'MED_IY_TBL_2016' in row:
        for d in row['MED_IY_TBL_2016']:
            s_code = d['s_tekiyo_code']
            k_code = d['k_tekiyo_code']
            if math.isnan(s_code) and math.isnan(k_code):
                continue
            
            code = s_code if not math.isnan(s_code) else k_code
            idx = x_dic[code]
            s = d['s_tensu']
            k = d['k_tensu']

            # 点数が負のとき、math.nanのときは 0
            x[idx] =\
                s if s>=0 else\
                k if k>=0 else\
                0

    # 診療に対応するindexに点数を挿入
    if 'MED_SI_TBL_2016' in row:
        for d in row['MED_SI_TBL_2016']:
            s_code = d['s_tekiyo_code']
            k_code = d['k_tekiyo_code']
            if math.isnan(s_code) and math.isnan(k_code):
                continue
            
            code = s_code if not math.isnan(s_code) else k_code
            idx = x_dic[code]
            s = d['s_tensu']
            k = d['k_tensu']
            
            # 点数が負のとき、math.nanのときは 0
            x[idx] =\
                s if s>=0 else\
                k if k>=0 else\
                0
    

    # 保険者レコードか公費レコード、どちらかに請求点数と決定点数に差があれば異常として1
    diff =  sum([d['diff_tensu'] for d in row['MED_HOKO_TBL_2016']])
    y = 1 if diff else 0
    
    xs.append(x)
    ys.append(y)
x_data = np.array(xs)
y_data = np.array(ys)

In [13]:
x_data.shape

(10000, 7211)

In [14]:
y_data.sum()

138

### NMF

In [15]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=666)

# resampling
rus = RandomUnderSampler(random_state=0)
ros = RandomOverSampler(random_state=0)
smt = SMOTE(random_state=0)
X_train_under, y_train_under = rus.fit_sample(X_train, y_train)
X_train_over, y_train_over = ros.fit_sample(X_train, y_train)
X_train_smote, y_train_smote = smt.fit_sample(X_train, y_train)

In [16]:
# modeling & evaluation
from sklearn.decomposition import NMF

pipe_gb = Pipeline([('nmf', NMF(n_components=2)),
                    ('clf', GradientBoostingClassifier(random_state=1))])

X_train, y_train = X_train_over, y_train_over
pipe_gb.fit(X_train, y_train)
print('Oversampling')
print('f1 score Train:', f1_score(y_train, pipe_gb.predict(X_train)))
print('f1 score Test:', f1_score(y_test, pipe_gb.predict(X_test)))
print('recall score Train:', recall_score(y_train, pipe_gb.predict(X_train)))
print('recall score Test:', recall_score(y_test, pipe_gb.predict(X_test)))
print()

X_train, y_train = X_train_smote, y_train_smote
pipe_gb.fit(X_train, y_train)
print('SMOTE')
print('f1 score Train:', f1_score(y_train, pipe_gb.predict(X_train)))
print('f1 score Test:', f1_score(y_test, pipe_gb.predict(X_test)))
print('recall score Train:', recall_score(y_train, pipe_gb.predict(X_train)))
print('recall score Test:', recall_score(y_test, pipe_gb.predict(X_test)))
print()


Original
f1 score Train: 0.721088435374
f1 score Test: 0.0869565217391
recall score Train: 0.563829787234
recall score Test: 0.0681818181818

Undersampling
f1 score Train: 0.978260869565
f1 score Test: 0.116169544741
recall score Train: 0.957446808511
recall score Test: 0.840909090909

Oversampling
f1 score Train: 0.931189710611
f1 score Test: 0.179640718563
recall score Train: 0.943527367507
recall score Test: 0.681818181818

SMOTE
f1 score Train: 0.882670594011
f1 score Test: 0.137404580153
recall score Train: 0.91123660585
recall score Test: 0.818181818182



In [18]:
A =  pipe_gb.predict(X_test)#*np.array([1,1])
tn, fp, fn, tp = confusion_matrix(y_test, A).ravel()
print('f1:',f1_score(y_test, A))
print('recall:',recall_score(y_test, A))
confusion_matrix(y_test, A),tn, fp, fn, tp

f1: 0.137404580153
recall: 0.818181818182


(array([[2512,  444],
        [   8,   36]]), 2512, 444, 8, 36)